# Counting Crowds with DL
## Proof Of Concept
The notebook will implement [Dense Scale Networks](https://arxiv.org/pdf/1906.09707.pdf) for the purpose of counting crowds for images.
The dataset will be the same as the one, detailed in the paper - the ShangaiTech.

## 1. Imports and loading the data.

In [6]:
import os
import pandas as pd
import numpy as np
import keras
import scipy.io
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.metrics import mse, mae

Using TensorFlow backend.


# 1.1 Getting Data to Train the model.
A couple of datasets are available to use for the purpose of training. My goal is to try to combine them all and train the dense scale network on all of them and evaluate them on all tests set.

*NOTE*: Provide List of more famous Datasets

### 1.1.1 UCF-QNRF_ECCV18

In [16]:
ucf_qnrf_example = scipy.io.loadmat('training_dataset/UCF-QNRF_ECCV18/Train/img_0001_ann.mat')
print(ucf_qnrf_example.keys())
print(ucf_qnrf_example['annPoints'].shape)

dict_keys(['__header__', '__version__', '__globals__', 'annPoints'])
(433, 2)


Judging from the keys of the example it seems, that the .mat files contain the annotation points. Since my gould is counting the people and not where they are I will take only the count of the annotations and not their respective coordinates.

In [29]:
def get_count_vector(dataset_path):
    '''
        Gets the density/count vector by reading the .mat file 
        and getting the shape of the file. 
        The function only requires the path to the dataset.
    '''
    file_contents = os.listdir(dataset_path)
    only_mat_files = list(filter(lambda x: '.mat' in x, file_contents))
    
    densities = {'image_name':[], 'count': []}
    for mat in only_mat_files:
        filepath = os.path.join(dataset_path, mat)
        mat_loaded = scipy.io.loadmat(filepath)
        densities['image_name'].append(mat.split('.mat')[0])
        densities['count'].append(mat_loaded['annPoints'].shape[0])      
    
    return densities

In [30]:
densities = get_count_vector('training_dataset/UCF-QNRF_ECCV18/Train/')
dens_df = pd.DataFrame(densities)

In [31]:
dens_df.head()

,image_name,count
0,img_0683_ann,928
1,img_0648_ann,260
2,img_0560_ann,108
3,img_0002_ann,121
4,img_0295_ann,697


## 2. Building the Model
In order to build ithe Deep scale network first the Dense Dialated Convolution Block has to be implemented.

In [2]:
class ConvBlock(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, activation='relu', padding=1, dilation_rate=1, **kwargs):
        super.__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(256, (1, 1), activation=activation)
        self.padding = keras.layers.ZeroPadding2D(padding=(padding, padding))
        self.conv2 = keras.layers.Conv2D(64, (3, 3), padding='valid', dilation_rate=dilation_rate, activation=activation)
    def call(self, inputs):
        out = self.conv1(inputs)
        out = self.padding(out)
        out = self.conv2(out)
        
        return out    

In [3]:
class DDCB(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.convBlock1 = ConvBlock()
        self.convBlock2 = ConvBlock(padding=2, dilation_rate=2)
        self.convBlock3 = ConvBlock(padding=3, dilation_rate=3)
        self.paddingOut = keras.layers.ZeroPadding2D(padding=(1,1))
        self.convOut = keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')
    
    def call(inputs):
        out1 = self.convBlock1(inputs)
        out2 = keras.layers.Concatenate([inputs, out1])
        out3 = self.convBlock2(out2)
        out4 = keras.layers.Concatenate([inputs, out1, out3])
        out5 = self.convBlock3(out4)
        out6 = keras.layers.Concatenate([inputs, out3, out5])
        out7 = self.paddingOut(out6)
        out8 = self.convOut(out7)
        return out8 

In [16]:
class DenseScaleNet(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, model=None, input_shape=None):
        super().__init__(**kwargs)
        if model is None:
            if input_shape is None:
                raise Exception('A model could not have an input shape set to None.')
                
            model = self.__create_backbone(input_shape)
        
        self.model = model
        self.DDCB1 = DDCB()
        self.DDCB2 = DDCB()
        self.DDCB3 = DDCB()
        self.padding1 = keras.layers.ZeroPadding2D(padding=(1,1))
        self.conv1 = keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')
        self.padding2 = keras.layers.ZeroPadding2D(padding=(1,1))
        self.conv2 = keras.layers.Conv2D(64, (3,3), padding='valid', activation='relu')
        self.outconv = keras.layers.Conv2D(1, (1, 1), activation='relu')
    
    def __create_backbone(self, input_shape):
        '''
            TODO: Add Doctstring
        '''
        vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', input_shape=input_shape)
        model = keras.models.Sequential()
        
        # we want to copy the first ten layers from VGG16
        for layer in vgg16_model[:11]:
            model.add(layer)
        
        return model
        
    def call(self, inputs):
        out1 = self.model(inputs)
        out2 = self.DDCB1(out1)
        out3 = out2 + out3
        out4 = self.DDCB2(out3)
        out5 = out4 + out2 + out1
        out6 = self.DDCB3(out5)
        out7 = out6 + out4 + out2 + out1
        out = self.padding1(out)
        out = self.conv1(out)
        out = self.padding2(out)
        out = self.conv2(out)
        out = self.outconv(out)
        
        return out           